## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [4]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진이 섞인 데이터셋이 있을 수 있어요.\n\n2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 분류할 수 있도록 인공지능의 '뇌' 역할을 하는 모델을 만듭니다. 이 모델은 수학적인 함수와 구조로 이루어져 있어요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 아무것도 모르기 때문에, 입력된 데이터를 바탕으로 예측을 해요.\n   - 그리고 그 예측이 얼마나 맞았는지(오차)를 계산해요.\n   - 오차가 크면, 모델이 잘못 배운 부분을 수정하기 위해 '학습'을 합니다. 이때, 오차를 줄이기 위해 가중치(모델 내부의 숫자 값들)를 조정하는 과정을 반복해요.\n   \n4. **반복과 개선**:\n   - 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 데이터를 이해하고 예측할 수 있게 돼요.\n   - 결국, 새로운 데이터가 들어와도 잘 맞출 수 있도록 학습이 완료됩니다.\n\n요약하자면, 인공지능 모델은 많은 데이터를 보고, 예측을 하고, 틀린 부분을 수정하는 과정을 반복하면서 점점 더 똑똑해지는 거예요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 345, 'prompt_tokens': 22, 'total_tokens': 367, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모읍니다. 예를 들어, 고양이 사진을 많이 보여주는 것처럼요.

2. **모델 설계**: 그 다음, 컴퓨터가 데이터를 이해하고 학습할 수 있도록 '모델'이라는 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 여러 층으로 이루어진 '신경망'일 수 있어요.

3. **학습 과정**:
   - 모델은 처음에는 아무것도 모른 채로 시작합니다.
   - 데이터를 하나씩 보여주면서, 모델이 내린 예측(예: 사진이 고양이인지 아닌지)이 맞는지 틀린지 알려줍니다.
   - 틀렸을 경우, 모델은 '오차'라는 값을 계산하고, 이 오차를 줄이기 위해 내부의 '가중치'라는 값을 조금씩 조정합니다.
   - 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 데이터를 이해하게 됩니다.

4. **검증과 개선**: 학습이 끝나면, 새로운 데이터(훈련에 사용하지 않은 데이터)를 통해 모델이 잘 작동하는지 테스트하고, 필요하면 더 개선합니다.

요약하자면, 인공지능 모델은 많은 데이터를 보고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정입니다. 이렇게 해서 우리가 원하는 작업(예를 들어, 사진 속의 물체를 알아맞히기)을 잘 수행할 수 있게 되는 거죠!

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [15]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 사진을 보고 고양이인지 강아지인지 구별하는 모델이라면, 다양한 고양이와 강아지 사진이 필요하겠죠.\n\n2. **모델 설계**: 그 다음, 인공지능이 어떤 구조로 작동할지 설계해요. 이 구조는 사람의 뇌와 비슷하게 여러 '뉴런'이 연결된 네트워크(신경망)일 수 있어요.\n\n3. **학습 과정**:\n   - **예측하기**: 모델은 처음에는 아무것도 모르기 때문에, 입력된 데이터를 보고 예측을 해요. 예를 들어, 사진이 고양이인지 강아지인지 맞추려고 해요.\n   - **오차 계산**: 그리고 모델이 만든 예측이 실제 정답(예를 들어, 사진이 고양이임)과 얼마나 다른지 '오차'라는 값을 계산해요.\n   - **수정하기**: 오차가 크면, 모델의 내부 값(가중치)을 조금씩 조정해서 다음에는 더 정확히 맞추도록 해요. 이 과정을 반복하면서 점점 더 잘 맞추게 되는 거죠.\n\n4. **반복 학습**: 이 과정을 여러 번 반복하면서, 모델은 점점 더 많은 데이터를 이해하고, 새로운 데이터에 대해서도 잘 예측할 수 있게 돼요.\n\n요약하자면, 인공지능은 많은 데이터를 보고, 예측을 하고, 오차를 줄이기 위해 계속 수정하는 과정을 반복하면서 '학습'하는 거예요. 이렇게 해서 점점 더 똑똑해지는 것이죠!"

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배울 수 있도록 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진을 모으는 거죠.

2. **모델 설계**: 그 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 정보를 처리하는 방식이에요.

3. **학습 과정**:
   - 모델이 처음에는 무작위로 예측하거나 판단을 해요.
   - 그 예측이 맞았는지 틀렸는지 '정답'과 비교해서 평가해요.
   - 틀렸다면, 왜 틀렸는지 원인을 찾고, 모델의 내부 값(파라미터)을 조금씩 수정해요.
   - 이 과정을 여러 번 반복하면서 점점 더 정확하게 예측하게 되는 거죠.

4. **최적화**: 이때 '손실 함수'라는 것을 사용해서, 얼마나 틀렸는지 수치로 계산하고, 이 값을 줄이기 위해 계속 수정하는 과정을 반복해요. 이 과정을 '학습' 또는 '훈련'이라고 부릅니다.

5. **완성**: 여러 번 반복해서 학습하면, 모델은 새로운 데이터에 대해서도 잘 예측할 수 있게 돼요. 예를 들어, 새로운 고양이 사진을 보여줬을 때 고양이인지 강아지인지 맞출 수 있게 되는 거죠.

요약하자면, 인공지능 모델은 많은 데이터를 보고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정으로 학습합니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [17]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [18]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [22]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
"Hi, I’d like to order a table for two, please."  
"Could I see the menu, please?"  
"I would like to order the grilled chicken and a side of salad."  
"Can I get that without any onions?"  
"Could I have the bill, please?"

- 한글 해석:  
"안녕하세요, 두 명 자리 예약하고 싶어요."  
"메뉴판 좀 볼 수 있을까요?"  
"그릴 치킨과 샐러드 하나 주문하고 싶어요."  
"양파 없이 부탁드릴 수 있을까요?"  
"계산서 주세요."


In [21]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello! I’d like to order a meal, please."  
"Hi! Could I get the menu, please?"  
"Can I have the [dish name], please?"  
"May I order the [specific dish], please?"  
"Could you recommend some popular dishes?"  

- 한글 해석:  
"안녕하세요! 음식을 주문하고 싶어요."  
"안녕하세요! 메뉴를 보여주실 수 있나요?"  
"[음식 이름] 하나 주세요."  
"저는 [특정 요리] 하나 주문할게요."  
"추천해주실 만한 인기 메뉴가 있나요?"

In [23]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
Customer: Hi, I’d like to order a large pepperoni pizza, please.
Staff: Sure! Would you like it delivery or pick-up?
Customer: Delivery, please. How long will it take?
Staff: It should arrive within 30 minutes. May I have your address?
Customer: Yes, it’s 123 Maple Street.
Staff: Thank you. Your total comes to $15.99. Can I get your phone number?
Customer: It’s 555-1234.
Staff: Great! Thank you for your order. It will be delivered soon.
Customer: Thank you very much!

- 한글 해석:
고객: 안녕하세요, 큰 페퍼로니 피자 하나 주문할게요.
직원: 네! 배달로 할까요, 아니면 포장하실 건가요?
고객: 배달로 주세요. 몇 분 걸릴까요?
직원: 30분 이내에 도착할 거예요. 주소를 알려주실 수 있나요?
고객: 네, 123 메이플 스트리트입니다.
직원: 감사합니다. 결제 금액은 15.99달러입니다. 전화번호도 여쭤봐도 될까요?
고객: 555-1234입니다.
직원: 좋아요! 주문해 주셔서 감사합니다. 곧 배달될 거예요.
고객: 정말 고마워요!